## Inference Pipeline

Now we have to predict the "future sales". This is a process that we are going to run every business day, in the middle of the night to try to predict how much each reseller is going to buy on their next pruchase. 

We will be basing on the max date of the dataset + 1 day because our extraction is not updated, but in production we can use the current day of the system and trust that we have all the relevant sales history from our transactional system. 

Note that to compute the features, now we only need the previous 30 days.


In [ ]:
import sagemaker
import boto3
from sagemaker.estimator import Estimator
from sagemaker.tuner import HyperparameterTuner
import numpy as np                                # For matrix operations and numerical processing
import pandas as pd                               # For munging tabular data
import os 
import time
from sagemaker.predictor import csv_serializer,RealTimePredictor
import datetime
import pickle
import awswrangler

In [ ]:
%store -r df

In [ ]:
%store -r df_r

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
max_date = df['date'].max()

In [ ]:
min_date = max_date - pd.Timedelta(days=30)

In [ ]:
df = df[(df['date'] > min_date)]

We are going to fill with amount 0 all the missing sales for each reseller every day.

In [ ]:
def completeItem(dfItem,max_date,min_date):
    r = pd.date_range(start=min_date, end=max_date)
    dfItemNew = dfItem.set_index('date').reindex(r).fillna(0.0).rename_axis('date').reset_index()
    dfItemNew['id_reseller'] = dfItem['id_reseller'].max()
    return dfItemNew


In [ ]:
dfCompletedList = []
for nid,item in df.groupby('id_reseller'):
    dfCompletedList.append(completeItem(item,max_date,min_date))
dfCompleted = pd.concat(dfCompletedList).copy()

In [ ]:
df = dfCompleted

In [ ]:
del dfCompleted

In [ ]:
df.head(10)

### Features for each reseller

In [ ]:
def complete_info(group):
    weekday = (max_date + pd.Timedelta(days=1)).weekday_name
    mean_last_30 = group['bill'].replace(0,np.nan).mean()
    std_last_30 = group['bill'].replace(0,np.nan).std()
    date_last_bill = group[group['bill'] != 0]['date'].max()
    days_without_purchase = (max_date + pd.Timedelta(days=1) - date_last_bill).days
    
    mean_last_7 = group[(group['date'] >= max_date - pd.Timedelta(days=6))]['bill'].replace(0,np.nan).mean()
    last_bill = group[group['bill'] > 0].sort_values('date',ascending=False).head(1)['bill'].values[0]
    return {'weekday':weekday,'mean-last-30':mean_last_30,
           'std-last-30':std_last_30,'mean-last-7':mean_last_7,'last_bill':last_bill, 
           'id_reseller':int(group['id_reseller'].max()), 'days_without_purchase':days_without_purchase}

In [ ]:
features = []
for index,group in df.groupby('id_reseller'):
    features.append(complete_info(group))

In [ ]:
df_features = pd.DataFrame(features)

In [ ]:
df_features.shape

### Merge with reseller info and compute dummy variables

In [ ]:
df_features.head()

In [ ]:
df_features = df_features.merge(df_r,how='inner',on='id_reseller')

In [ ]:
df_features.shape

In [ ]:
df_features['zone'] = df_features['zone'].apply(lambda x: x if x in [1019,1050,1031,1033,1051,1067] else 0)

In [ ]:
pickle_in = open("preprocessing.pkl","rb")
pipe_list = pickle.load(pickle_in)
# [le_cluster,ohe_cluster,le_zone,ohe_zone,le_weekday,ohe_weekday]

In [ ]:
df_cluster = pd.DataFrame(
    pipe_list[1].transform(pipe_list[0].transform(df_features['cluster']).reshape(-1, 1)).todense()
)
df_cluster = df_cluster.add_prefix('cluster_')

In [ ]:
df_zone = pd.DataFrame(
    pipe_list[3].transform(pipe_list[2].transform(df_features['zone']).reshape(-1, 1)).todense()
)
df_zone = df_zone.add_prefix('zone_')

In [ ]:
df_weekday = pd.DataFrame(
    pipe_list[5].transform(pipe_list[4].transform(df_features['weekday']).reshape(-1, 1)).todense()
)
df_weekday = df_weekday.add_prefix('weekday_')

In [ ]:
df_to_predict = pd.concat([df_features,df_cluster,df_zone,df_weekday],axis=1)

### Re-order features
 Now we have to make sure that the features are in the same order we used for training and that we don't have any extra columns.


### Here you are going to need the same columns and order that it's displayed in notebook PROD1 

In [ ]:
df_to_predict.columns

In [ ]:
%store -r pred_columns

In [ ]:
df_to_predict_feats = df_to_predict[pred_columns]

In [ ]:
df_to_predict_feats.to_csv('to_predict.csv',header=False,index=False)

In [ ]:
df_to_predict[['id_reseller']].to_csv('id_reseller_to_predict.csv',header=False,index=False)